In [1]:
import pandas as pd
import numpy as np

%pylab inline

Populating the interactive namespace from numpy and matplotlib


How many taxi trips are reported in the original file?

In [2]:
%%sh
wc -l yellow_tripdata_2016-12.csv 

wc: yellow_tripdata_2016-12.csv: open: No such file or directory


Getting random samples:

In [3]:
%%sh
head -n 1 yellow_tripdata_2016-12.csv > sample100.csv
tail -n +2 yellow_tripdata_2016-12.csv | gshuf -n 100 | sed 's/,,//g' >> sample100.csv

head: yellow_tripdata_2016-12.csv: No such file or directory
tail: yellow_tripdata_2016-12.csv: No such file or directory
sh: line 2: gshuf: command not found


In [5]:
sample100 = pd.read_csv('sample100.csv', parse_dates=[1,2])
sample100.head() 

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-12-15 12:18:37,2016-12-15 12:28:45,6,0.50,1,N,141,237,2,7.5,0.0,0.5,0.00,0.00,0.3,8.30
1,1,2016-12-20 22:39:23,2016-12-20 22:50:27,1,2.00,1,N,246,164,1,9.5,0.5,0.5,3.00,0.00,0.3,13.80
2,1,2016-12-09 23:02:06,2016-12-09 23:11:03,1,1.10,1,N,114,79,1,7.5,0.5,0.5,1.75,0.00,0.3,10.55
3,1,2016-12-11 15:40:19,2016-12-11 16:09:53,1,11.90,1,N,138,224,1,34.0,0.0,0.5,8.05,5.54,0.3,48.39
4,2,2016-12-19 11:34:28,2016-12-19 11:47:51,1,1.96,1,N,261,144,2,10.5,0.0,0.5,0.00,0.00,0.3,11.30


## Estimating the proportion of tippers

In [6]:
is_tipped = sample100.tip_amount>0
is_tipped.mean()

0.66

Standard deviation:

In [7]:
ph = is_tipped.mean()
s = np.sqrt(ph * (1-ph) / len(is_tipped))
s

0.04737087712930804

95% confidence interval:

In [8]:
from statsmodels.stats.proportion import proportion_confint
proportion_confint(sum(is_tipped), len(is_tipped), alpha=0.05)

ModuleNotFoundError: No module named 'statsmodels'

Pretty wide! How big do we need a sample for the 95% confidence interval to be approximately 2% wide?

In [ ]:
from statsmodels.stats.proportion import samplesize_confint_proportion
int(np.ceil(samplesize_confint_proportion(ph, 0.01)))

Let's take a bigger sample:

In [ ]:
%%sh
head -n 1 yellow_tripdata_2016-12.csv > sample10000.csv
tail -n +2 yellow_tripdata_2016-12.csv | gshuf -n 10000 | sed 's/,,//g' >> sample10000.csv

In [ ]:
sample10000 = pd.read_csv('sample10000.csv', parse_dates=[1,2])
is_tipped = sample10000.tip_amount>0
is_tipped.mean()

In [ ]:
ph = is_tipped.mean()
s = np.sqrt(ph * (1-ph) / len(is_tipped))
s

In [ ]:
proportion_confint(sum(is_tipped), len(is_tipped), alpha=0.05)

It is indeed about 2% wide. 

## Estimating the average trip duration

In [ ]:
sample100['duration'] = [x.total_seconds() / 60 for x in sample100.tpep_dropoff_datetime -  sample100.tpep_pickup_datetime]

In [ ]:
sample100['duration'].mean() 

In [ ]:
s = sample100['duration'].std(ddof=1) / np.sqrt(len(sample100['duration']))
s

In [ ]:
from statsmodels.stats.weightstats import _tconfint_generic
_tconfint_generic(sample100['duration'].mean(), 
                  s, 
                  len(sample100['duration']) - 1, 
                  0.05, 'two-sided')

In [ ]:
sample100.hist(column = 'duration', bins=30, normed=True)

In [ ]:
sample10000['duration'] = [x.total_seconds() / 60 for x in sample10000.tpep_dropoff_datetime -  sample10000.tpep_pickup_datetime]

In [ ]:
sample10000['duration'].mean()

In [ ]:
s = sample10000['duration'].std(ddof=1) / np.sqrt(len(sample10000['duration']))
s

In [ ]:
_tconfint_generic(sample10000['duration'].mean(), 
                  s, 
                  len(sample10000['duration']) - 1, 
                  0.05, 'two-sided')

In [ ]:
sample10000.hist(column = 'duration', bins=30, normed=True)

In [ ]:
tmp = sample10000['duration'] < 120
tmp.value_counts()

In [ ]:
sample10000[sample10000['duration'] < 120].hist(column = 'duration', bins=30, normed=True)
sample10000['duration'][sample10000['duration'] < 120].mean()

In [ ]:
sample100['duration'].median()

In [ ]:
sample10000['duration'].median()

In [ ]:
sample10000['duration'][sample10000['duration'] < 120].median()

Bootstrap:

In [ ]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
    
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [ ]:
median_duration = list(map(np.median, get_bootstrap_samples(sample100['duration'].values, 1000)))
stat_intervals(median_duration, 0.05)

In [ ]:
median_duration = list(map(np.median, get_bootstrap_samples(sample10000['duration'].values, 1000)))
stat_intervals(median_duration, 0.05)